## Overview of the whole project goal and step by step map
### ...

### Importing nessessary libraries

In [91]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm, t

### Loading 2 years of closing prices from Jan 2023 until Jan 2025 for 5 assets from yahoo finance and droping non trading days

In [92]:
tickers = ['AAPL', 'JNJ', 'XOM', 'JPM', 'WMT']

data = yf.download(tickers, start='2023-01-01', end='2025-01-02', interval='1d')['Close']

data = data.dropna()

closing_prices = data[-501:]

print(closing_prices)

/var/folders/g3/d8cr9sbj1nq0g6fwxyhphc380000gn/T/ipykernel_24929/3224209059.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start='2023-01-01', end='2025-01-02', interval='1d')['Close']
[*********************100%***********************]  5 of 5 completed

Ticker            AAPL         JNJ         JPM        WMT         XOM
Date                                                                 
2023-01-04  124.744125  166.685318  127.278275  46.401100   98.097069
2023-01-05  123.421257  165.454590  127.250038  46.242950  100.291893
2023-01-06  127.962395  166.796356  129.685059  47.375858  101.504105
2023-01-09  128.485626  162.474930  129.149139  46.785194   99.612320
2023-01-10  129.058258  162.086273  130.305542  46.756153  101.100052
...                ...         ...         ...        ...         ...
2024-12-24  257.578674  143.461914  238.440506  92.203255  104.494308
2024-12-26  258.396667  143.196320  239.257263  92.312691  104.582695
2024-12-27  254.974930  142.675018  237.318726  91.188515  104.572884
2024-12-30  251.593094  140.992996  235.498260  90.104111  103.865776
2024-12-31  249.817383  142.252045  235.882050  89.885246  105.643356

[501 rows x 5 columns]


### Transforming closing prices to logarithmic returns

In [113]:
log_returns = np.log(closing_prices/closing_prices.shift(1)).dropna()

print(log_returns)

Ticker          AAPL       JNJ       JPM       WMT       XOM
Date                                                        
2023-01-05 -0.010661 -0.007411 -0.000222 -0.003414  0.022127
2023-01-06  0.036133  0.008077  0.018955  0.024204  0.012014
2023-01-09  0.004081 -0.026250 -0.004141 -0.012546 -0.018813
2023-01-10  0.004447 -0.002395  0.008914 -0.000621  0.014825
2023-01-11  0.020892 -0.001600  0.007404  0.008729  0.011560
...              ...       ...       ...       ...       ...
2024-12-24  0.011413  0.003985  0.016310  0.025462  0.000940
2024-12-26  0.003171 -0.001853  0.003420  0.001186  0.000845
2024-12-27 -0.013331 -0.003647 -0.008135 -0.012253 -0.000094
2024-12-30 -0.013352 -0.011859 -0.007701 -0.011963 -0.006785
2024-12-31 -0.007083  0.008890  0.001628 -0.002432  0.016969

[500 rows x 5 columns]


### Setting the variables for window length, confidence level, normal quantiles and portfolio weights

In [95]:
window = 250

alpha_95 = 0.95
alpha_99 = 0.99

z_score_95 = norm.ppf(alpha_95)
z_score_99 = norm.ppf(alpha_99)

weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

### Calculating realized portfolio returns for testing window

In [98]:
realized_returns = log_returns @ weights
realized_returns = realized_returns[window:]

### Parameric rolling window approach for portfolio VaR prediction

In [96]:
par_rolling_var_95 = []
par_rolling_var_99 = []

for t in range(window, len(log_returns)):
    window_log_returns = log_returns.iloc[t-window:t]
    cov_matrix = window_log_returns.cov().values
    portfolio_std = np.sqrt(weights.T @ cov_matrix @ weights)
    var_95 = z_score_95 * portfolio_std
    var_99 = z_score_99 * portfolio_std
    par_rolling_var_95.append(var_95) # assume mean = 0 due to negligible effect in daily log returns
    par_rolling_var_99.append(var_99)

### Aligning the forecasted VaR values with realized returns series for 5 assets 

In [97]:
par_var_95_series = pd.Series(par_rolling_var_95, index=log_returns.index[window:])
par_var_99_series = pd.Series(par_rolling_var_99, index=log_returns.index[window:])

### Calculating number VaR violation rates for 95% and 99% confidence level 

In [99]:
par_breaches_95 = realized_returns < -par_var_95_series
par_breaches_99 = realized_returns < -par_var_99_series

par_breach_rate_95 = par_breaches_95.sum()/len(par_breaches_95)
par_breach_rate_99 = par_breaches_99.sum()/len(par_breaches_99)

print(f'breach_rate_95: {par_breach_rate_95:.2%}')
print(f'breach_rate_99: {par_breach_rate_99:.2%}')

breach_rate_95: 4.00%
breach_rate_99: 1.60%


### Performing Kupiec test (Proportions of Faliure) for 95% and 99% confidence level

In [100]:
from scipy.stats import chi2

T = len(par_var_95_series)
x = len(par_breaches_95)
p = alpha_95  # nominal exception rate, e.g. 0.05

hat_p = x / T
L0 = ((1-p)**(T-x)) * (p**x)
L1 = ((1-hat_p)**(T-x)) * (hat_p**x)
LR_pof = -2 * np.log(L0 / L1)
p_value = 1 - chi2.cdf(LR_pof, df=1)

print(p_value)

4.1000723660644667e-07


In [101]:
T = len(par_var_99_series)
x = len(par_breaches_99)
p = alpha_99  

hat_p = x / T
L0 = ((1-p)**(T-x)) * (p**x)
L1 = ((1-hat_p)**(T-x)) * (hat_p**x)
LR_pof = -2 * np.log(L0 / L1)
p_value = 1 - chi2.cdf(LR_pof, df=1)
print(p_value)

0.024981503053449705


### Historical rolling window approach for portfolio VaR prediction

In [102]:
hist_rolling_var_95 = []
hist_rolling_var_99 = []

port_log_returns = log_returns @ weights

for t in range(window, len(port_log_returns)):
    window_t = port_log_returns[t-window:t]
    var_t_95 = np.quantile(window_t, alpha_95)
    var_t_99 = np.quantile(window_t, alpha_99)
    hist_rolling_var_95.append(var_t_95)
    hist_rolling_var_99.append(var_t_99)

### Aligning the forecasted VaR values with realized returns series for 5 assets 

In [103]:
hist_var_95_series = pd.Series(hist_rolling_var_95, index=log_returns.index[window:])
hist_var_99_series = pd.Series(hist_rolling_var_99, index=log_returns.index[window:])

### Calculating number VaR violation rates for 95% and 99% confidence level 

In [104]:
hist_breaches_95 = realized_returns < -hist_var_95_series
hist_breaches_99 = realized_returns < -hist_var_99_series

hist_breaches_rate_95 = hist_breaches_95.sum()/len(hist_breaches_95)
hist_breaches_rate_99 = hist_breaches_99.sum()/len(hist_breaches_99)

print(f'breach_rate_95: {hist_breaches_rate_95:.2%}')
print(f'breach_rate_99: {hist_breaches_rate_99:.2%}')

breach_rate_95: 4.00%
breach_rate_99: 1.20%


### Performing Kupiec test (Proportions of Faliure) for 95% and 99% confidence level

In [105]:
from scipy.stats import chi2
import numpy as np

T = len(hist_var_95_series)           # total number of observations
x = len(hist_breaches_95)         # number of VaR breaches (exceptions)
p = alpha_95                     # expected failure rate, e.g. 0.05

# Empirical exception rate
hat_p = x / T

# Avoid log(0) and division by zero
if hat_p == 0 or hat_p == 1:
    LR_pof = np.inf  # test is undefined in this case
    p_value = 0.0
else:
    # Kupiec likelihood ratio statistic
    L0 = (1 - p)**(T - x) * p**x
    L1 = (1 - hat_p)**(T - x) * hat_p**x
    LR_pof = -2 * np.log(L0 / L1)
    p_value = 1 - chi2.cdf(LR_pof, df=1)

print("LR PoF statistic:", LR_pof)
print("p-value:", p_value)


LR PoF statistic: inf
p-value: 0.0


In [106]:
from scipy.stats import chi2
import numpy as np

T = len(hist_var_99_series)           # total number of observations
x = len(hist_breaches_99)         # number of VaR breaches (exceptions)
p = alpha_99                     # expected failure rate, e.g. 0.01

# Empirical exception rate
hat_p = x / T

# Avoid log(0) and division by zero
if hat_p == 0 or hat_p == 1:
    LR_pof = np.inf  # test is undefined in this case
    p_value = 0.0
else:
    # Kupiec likelihood ratio statistic
    L0 = (1 - p)**(T - x) * p**x
    L1 = (1 - hat_p)**(T - x) * hat_p**x
    LR_pof = -2 * np.log(L0 / L1)
    p_value = 1 - chi2.cdf(LR_pof, df=1)

print("LR PoF statistic:", LR_pof)
print("p-value:", p_value)

LR PoF statistic: inf
p-value: 0.0


### Monte Carlo rolling window approach with historical covariance matrix for portfolio VaR prediction

In [107]:
n_of_simulations = 10_000

mc_rolling_var_95 = []
mc_rolling_var_99 = []

for t in range(window, len(log_returns)):

    window_log_returns = log_returns.iloc[t-window:t]

    mu = window_log_returns.mean().values
    cov_matrix = window_log_returns.cov().values

    normal_simulation = np.random.multivariate_normal(mu, cov_matrix, n_of_simulations)
    sim_portfolio_returns = normal_simulation @ weights.T

    var_95 = -np.percentile(sim_portfolio_returns, 5)
    var_99 = -np.percentile(sim_portfolio_returns, 1)
    mc_rolling_var_95.append(var_95)
    mc_rolling_var_99.append(var_99)

### Aligning the forecasted VaR values with realized returns series for 5 assets 

In [108]:
mc_var_95_series = pd.Series(mc_rolling_var_95, index=log_returns.index[window:])
mc_var_99_series = pd.Series(mc_rolling_var_99, index=log_returns.index[window:])

### Calculating number VaR violation rates for 95% and 99% confidence level 

In [109]:
mc_breaches_95 = realized_returns < -mc_var_95_series
mc_breaches_99 = realized_returns < -mc_var_99_series

mc_breach_rate_95 = mc_breaches_95.sum()/len(mc_breaches_95)
mc_breach_rate_99 = mc_breaches_99.sum()/len(mc_breaches_99)

print(f'breach_rate_95: {mc_breach_rate_95:.2%}')
print(f'breach_rate_99: {mc_breach_rate_99:.2%}')

breach_rate_95: 5.20%
breach_rate_99: 2.00%


### Performing Kupiec test (Proportions of Faliure) for 95% and 99% confidence level

In [110]:
from scipy.stats import chi2

T = len(mc_var_95_series)
x = len(mc_breaches_95)
p = alpha_95  # nominal exception rate, e.g. 0.05

hat_p = x / T
L0 = ((1-p)**(T-x)) * (p**x)
L1 = ((1-hat_p)**(T-x)) * (hat_p**x)
LR_pof = -2 * np.log(L0 / L1)
p_value = 1 - chi2.cdf(LR_pof, df=1)

print(p_value)

4.1000723660644667e-07


In [111]:
T = len(mc_var_99_series)
x = len(mc_breaches_99)
p = alpha_99

hat_p = x / T
L0 = ((1-p)**(T-x)) * (p**x)
L1 = ((1-hat_p)**(T-x)) * (hat_p**x)
LR_pof = -2 * np.log(L0 / L1)
p_value = 1 - chi2.cdf(LR_pof, df=1)

print(p_value)

0.024981503053449705
